In [2]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import pickle
import time
import re
import nltk
import sys
import html
import xml.sax.saxutils as saxutils
from html.parser import HTMLParser
from io import StringIO
import random
import operator
import string
import gensim

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from warnings import filterwarnings
filterwarnings('ignore')


from transformers import BertTokenizer, BertModel
from transformers import DistilBertModel,DistilBertTokenizer

from scipy.spatial.distance import cosine

from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
Stem=stemmer.stem

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
lemm=wordnet_lemmatizer.lemmatize

import spacy
nlp = spacy.load("en_core_web_sm")

from nltk import ngrams

import dataframe_image as dfi

from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
#tknzr.tokenize(text)

In [3]:
from gensim.models.fasttext import load_facebook_model

import fasttext.util
#fasttext.util.download_model('en', if_exists='ignore')  # English

In [4]:
#print(len(cachedStopWords))
#print(len(cachedStopWords))
#print(type(cachedStopWords))

nltk_stopwords=set(cachedStopWords)

english_alghabet=['b','c','e','f','g','h','j','k','l','n','p','q','r','u','v','w','x','z']

numbers_remove=['one','two','three','four','five','six','seven','eight','nine','ten','tens','twenty',
                'fourty','fifty','sixty','seventy','eighty','ninety','hundred','hundreds','million','billion','trillion',
                'millions','thousand','thousands','second','third','forth','tenth','billions','trillions'] 

miscellaneous_remove=['absolutely', 'actually', 'adieu', 'ain', "ain't", 'aint', 'almost',
                       'awesome','awfully','amazing','interesting',
                       'alright','alrighty', 'amoungst', 'anybody', 'anymore', 'anyways', 'apart', 'apparently', 'anytime',
                       'appropriate',  'approximately', 'arent', 'behold', 'better', 'bravo','briefly','bad','best','brilliant',
                       'bye', 'cant', 'certainly', 'chrissakes', 'clearly', 'completely',
                       'congrat', 'congrats','congratulation', 'congratulations', 'consequently', 'cool', 'couldnt',
                       'darnit', 'de','dear', 'definitely','disappointing', 'didn', 'doesn', 'don', 'downwards',
                       'disgusting','dude','down','eg',"e.g.",'i.e.',
                       'encore','entirely', 'especially', 'et', 'etc', 'everybody', 'ex', 'exactly', 'excellent',
                       'fantastic','far', 'farewell','funny',
                       'felicitation', 'felicitations','finally', 'fully','furthermore', 'gadzooks', 
                       'good', 'goodby','goodness', 'gracious', 'great', 
                       'greetings', 'hallo', 'hardly', 'hasnt', 'haven', 'hello', 'here','hi', 'hither','higher','hopefully',
                       'here','there','including',
                       'howbeit', 'ie', 'immediately', 'inasmuch', 'inner', 'insofar', 'instead', 'inward', 'important',
                       'indeed','just', "it'd", "it'll", 'inside','kertyschoo', 'kg', 'km', 'lackaday', 
                       'largely', 'lately', 'later','lovely',
                       'lest', 'let', 'lets', 'likely', 'little', 'ltd', 'lower','magnificent', 'mainly', 'marvelous',
                       'myself','yourself','yourselves','himself','herself','hisself','ourselves','themsleves',
                       'maybe', 'meantime', 'merely', 'minus', 'near', 'nearly', 'necessary', 'never', 
                       'non', 'normally', 'obviously', 'ok', 'okay', 'ones', 'outside', 'over','other','others','only',
                       'overall', 'particular', 'particularly', 'please', 'plus', 'poorly', 'possible','up',
                       'possibly', 'potentially', 'predominantly', 'presumably', 'previously','primarily', 'probably',
                       'promising',
                       'promptly', 'readily', 'really', 'reasonably', 'recent', 'recently', 'ref',
                       'refs', 'regardless', 'related', 'relatively', 'respectively', 'resulting', 'right', 'sec', 
                       'secondly','self', 'selves', 'seriously', 'shall', 'shucks','somebody', 'somethan','sorry',
                       'somewhat', 'soon', 'late' , 'sorry', 'stupid', 'sub', 'substantially', 'successfully', 'sufficiently',
                       
                       'super', 'sure', "t's", 'th', 'thank', 'thanks', 'thanx', "that've", 'thats', 'there', "there'll",
                       "there've", 'thered', 'thereof', 'therere', 'theres', 'thereto', 'theyd', 'theyre', 'thorough',
                       'then','thankfully','too','today','yesterday','tomorrow','night',"morning",'afternoon','noon','tonight',
                       'evening','day','everyday', 'everynight','todays','nights','mornings','noons','afternoons','days',
                       'evenings','week','month','year',
                       'thoroughly', 'tnx', 'too','truly', 'twice', 'undoubtedly','unfortunately', 'unlike','unlikely',
                       'unto',  'usually', 'vs', 'welcome', 'well', 'went', 'werent', 'what', 'whatever', 'wheres', 'widely',
                       'wonderful', 'wont', 'wouldnt', 'wrong', 'worst','worse','www', 'yes', 'youd', 'youre', 'yummy', 
                       'zoinks','shit','literally','literal','pleasure','effective','fabulous','delighted',
                       'saturday','sunday','monday','tuesday','wednesday','thursday', 'friday','past','future','suitable',
                       'much','many','less','least','few','lots','lot','fewer','fewset','therefore','pm',
                       'afaik', 'br', 'idk','smh','qotd', 'ftw','bfn','yw', 'icymi','fomo','smdh', 'b4','imho',
                       'urdddd','fab' ,'delightful','absolute','pleasure','huge','latest','nowadays',
                       'january','february','april','june','july','august','september','october',
                       'november','december', 'autumn' ,'spring','winter','summer',
                       'mr','madam','sir','mrs','easy', 'difficult','dr',
                       'weekend','south','north','west','east','asia','africa','europe','america','totally',
                       'come', 'comes', 'coming', 'came', 'seems', 'gives', 'gave', 'makes', 'made', 'keeps', 'kept', 
                       'calls', 'called', 'says', 'saying', 'said', 'goes', 'went', 'gone', 'got', 'saw', 'seen', 'shows',
                       'shown', 'took', 'taken', 'uses', 'moved', 'moves', 'puts',
                       'using','seem','give','make','keep','call','say','go','get','see','seems','seeming',
                       'seemed','show','take','made','used','move','become','became','becoming','becomes','put','use',
                       'find', 'finds', 'finding','aka',
                       'lol' , 'brb', 'lmk', 'ama', 'tbh', 'irl', "tl;dr", 'fml', 'bfn' ,' br', 'ht', "hth",'j/k', 'lmao' ] #cool
                        # small big large useful 

interjection_remove=['aaaahh', 'aaah', 'aaargh', 'aaay', 'aagh', 'aah',
                   'aargh', 'achoo', 'adios', 'ah', 'aha', 'ahem', 'ahh', 'ahhh',
                   'ahoy', 'alas', 'allo', 'amen', 'areet', 'argh', 'arrggh',
                   'arrividerci', 'asap', 'attaboy', 'avaunt', 'aw', 'aw', 'aww',
                   'awww', 'ay', 'ay', 'aye', 'ayeaugh', 'bada', 'badum', 'bah',
                   'bahaha', 'bam', 'bazinga', 'behold', 'bingce', 'bingo', 'blah',
                   'blech', 'bleh', 'blimey', 'bonjour', 'boo', 'booh', 'boohoo',
                   'booyah', 'bravo', 'brr', 'brrrr', 'btw', 'bwahaha', 'capeesh',
                   'capisce', 'cheerio', 'cheers', 'ciao', 'cor', 'cowabunga',
                   'crikey', 'cripes', 'da', 'dabba', 'dah', 'dammit', 'damn', 'dang',
                   'darn', 'de', 'dee', 'di', 'dizamn', 'doh', 'doo', 'drat', 'duh',
                   'dum', 'eeeek', 'eek', 'eep', 'egad', 'egads', 'eh', 'ehem', 'em',
                   'er', 'eureka', 'eww', 'ewww', 'eyh', 'fiddledeedee', 'fie',
                   'fore', 'foul', 'fuff', 'gah', 'gak', 'gee', 'geez', 'gesundheit',
                   'giddyap', 'golly', 'gosh', 'grr', 'grrrr', 'ha', 'hah', 'haha',
                   'hahaha', 'hallelujah', 'halloa', 'harrumph', 'harumph', 'haw',
                   'heck', 'heck', 'heeey', 'heh', 'hehe', 'hey', 'hhh', 'hic', 'hm',
                   'hmm', 'hmmm', 'hmmmm', 'hmmph', 'hmpf', 'ho', 'hola', 'hoo',
                   'hooray', 'howdy', 'hrmm', 'hrmph', 'hrmph', 'hrrmph', 'hu', 'huh',
                   'hullo', 'humph', 'hurrah', 'huzza', 'huzzah', 'ich', 'ick',
                   'ixnay', 'jeepers', 'jeez', 'kaboom', 'kapow', 'kerwham', 'la',
                   'lala', 'lo', 'lordy', 'meh', 'mhm', 'ml', 'mm', 'mmh', 'mmhm',
                   'mmm', 'muahaha', 'mwah', 'mwahaha', 'na','nay','nah', 'nanu', 'nooo', 'nope',
                   'nuh', 'oh', 'ohh', 'oho', 'oi', 'okeydoke', 'om', 'oof', 'ooh',
                   'oomph', 'oooh', 'ooooh', 'oops', 'ouch', 'ow', 'oww', 'oy',
                   'oyez', 'oyh', 'pew', 'pff', 'pffh', 'pfft', 'phew', 'phut',
                   'phweep', 'phwoar', 'phwoarr', 'poof', 'poogh', 'prethee',
                   'prithee', 'prosit', 'pssh', 'psst', 'queep', 'roger', 'salaam',
                   'salam', 'sheesh', 'shh', 'shhh', 'shitfire', 'shoo', 'shoop',
                   'shush', 'sigh', 'sssh', 'strewth', 'ta', 'tarnations', 'tchah',
                   'teehee', 'tish', 'touché', 'tsk', 'tss', 'tut', 'uggh', 'ugh',
                   'uh', 'uhh', 'uhm', 'um', 'umm', 'ummm', 'umph', 'unh', 'upadaisy',
                   'upsadaisy', 'ur', 'urgh', 'vay', 'vayf', 'viva', 'voila', 'waa',
                   'waaaaah', 'waah', 'wah', 'wahey', 'wassup', 'weee', 'welp',
                   'wham', 'whamo', 'whee', 'whew', 'whizz', 'whoa',
                   'whoo', 'whoopee','whoop', 'whoops', 'whoopsy', 'whoosh', 'woah', 'woo',
                   'woohoo', 'wotcha', 'wotcher', 'wow', 'wowsers', 'wowsers',
                   'wuzzup', 'wuzzup', 'wuzzup', 'ya', 'yabba', 'yada', 'yadda',
                   'yak', 'yarooh', 'yay', 'yea', 'yeah', 'yech', 'yee', 'yeeeeaah',
                   'yeehaw', 'yeow', 'yes', 'yessiree', 'yew', 'yikes', 'yippee',
                   'yo', 'yoo', 'yoohoo', 'yow', 'yowza', 'yuck', 'yuh', 'zing',
                   'zoiks', 'zomfg', 'zomg', 'zounds', 'zut']
             
import spacy
sp = spacy.load('en_core_web_sm')
spacy_stopwords = sp.Defaults.stop_words
type(spacy_stopwords)
#spacy_exclude=['using','name','seem','give','make','keep','call','say','go','get','see','seems','seeming',
#               'seemed','show','take','made','used','move','become','became','becoming','becomes','put','use']# serious

from stop_words import get_stop_words
stop_words = get_stop_words('en')
stop_words1 = get_stop_words('english')
#print(type(stop_words1))
#print()
#print(stop_words1)
lib_stopwords=set(stop_words1)

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
#print(type(ENGLISH_STOP_WORDS))
#print()
#print(set(ENGLISH_STOP_WORDS))
#sklearn_exclude=['find','get','found','go','see','seem','seems','give','seemed','take','keep','show','put','made'] # system  cry
sklearn_stopwords=set(ENGLISH_STOP_WORDS)

#spacy_stopwords.difference_update(set(spacy_exclude))
#sklearn_stopwords.difference_update(set(sklearn_stopwords))
#for removing "just" one item, use "remove"
temp_1=set([])
#######################
temp_1.update(nltk_stopwords)
temp_1.update(lib_stopwords)
temp_1.update(sklearn_stopwords)
temp_1.update(spacy_stopwords)
temp_1.update(set(english_alghabet))
temp_1.update(set(numbers_remove)) 
temp_1.update(set(miscellaneous_remove))
temp_1.update(set(interjection_remove))
temp_1.update(['rt','be','will','was','were','is','am','are','have','has','had','do','does','done'])
###              ,'thing','want','getting', 'looking','way'])
###temp_1.update(['rt','like','look','get','take'])
cachedStopWords=temp_1
len(cachedStopWords)

1134

In [5]:
type(cachedStopWords)

set

In [6]:
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [7]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

#print(lemmatizer.lemmatize("I am loving it")) #I am loving it
#print(lemmatizer.lemmatize("loving")) #loving
#print(lemmatizer.lemmatize("loving", "v")) #love
#print(lemmatize_sentence("I am loving it")) #I be love it

In [8]:
def cleaning (text):
    
    #order of lines is important
    
    text=strip_tags(text)
    #text=html.unescape(text)   # stripping or converting html entities 
    #text=saxutils.unescape(text)
    
    #convertings words that their lower and uper cases are different
    text=re.sub(" US | U\.S\. ", ' USA ', text) # before lower 
    
    #converting
    text = re.sub("“|”", '', text)  #before next lines
    text = re.sub("’|′|‘|`", "'", text)  #before next lines
    
    #removing tabs and lines
    text=re.sub('\t|\n', ' ', text)
    
    #converting lower_case
    text = text.lower() 
    
    #converting
    text=re.sub('\$|£|€|¥|dollar|dollars|yen|yens|euros', ' money ', text)   # not euro 
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    
    #removing emoji
    text = emoji_pattern.sub(r' ', text) 

    #removing emojis and non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+|,Ä¶',' ', text)  
    
    #removeing http and https (URL)
    text = re.sub(r'(http://|https://)\S+', '', text)
    
    #removing www (URL)
    text=re.sub(r'www\.\S+', '', text)
    
    #removing targets
    text=re.sub('( |^)@\S+', '', text) 

    #removing common expressions
    
    text=re.sub("looking forward to|look forward to|make sure|kidding me|\
                |in my opinion|by the way,|as soon as possible|shaking my head|i don't know|I do not know|\
                |in real life|quote of the day|as far as i know|shake my head|\
                |to be honest|in other words|let me know|just kidding|hope that helps|hat tip|\
                |just like that|happy birthday|never mind|well-done|\
                |in my humble opinion|happy new year|you're welcome|you are welcome| \
                |it doesn't matter|it does not matter|i think|i wonder|do you think", ' ', text) 
    
    
    #convertings
    text=re.sub("can't", 'cannot', text) # before other n't 
    text=re.sub("can not ", 'cannot ', text)  
    text=re.sub("'ve",' have', text)
    text=re.sub("n't",' not', text)
    text=re.sub("'ll",' will', text)
    #text=re.sub("'d",' would', text)
    text=re.sub("'re",' are', text)
    text=re.sub("i'm",'i am', text)
    #text=re.sub("&",' and ', text)
    text=re.sub(" w/ ",' with ', text)
    text=re.sub(" w/i | w/in ",' within ', text)
    text=re.sub(" w/o ",' without ', text)
    text=re.sub(" c/o ",' care of ', text)
    text=re.sub(" h/t ",' hat tip ', text)
    text=re.sub(" b/c ",' because ', text)
    #text=re.sub("=",' equals to ', text)
    text=re.sub("=",' = ', text)
    #text=re.sub("\+",' + ', text)
    #text=re.sub("\+",' plus ', text)
    text=re.sub("united states",'usa', text)
    text=re.sub("united kingdom",'uk', text)
    text=re.sub(" the us ",' usa ', text)
    text=re.sub("start-up|start_up",'startup', text)
    text=re.sub("u\.s\.a", 'usa', text)  #try text=re.sub("u.s.a", 'usa', text) with text=substantially 
    text=re.sub("aka", 'also known as', text)     
    text=re.sub("'"," ' ", text)     
    
    text= re.sub("(\?)+", '? ',text)     
    text= re.sub("(!)+", '! ',text)     
    text= re.sub("(\.\.)+", ' ',text)   

#    text = "".join(lemmatize_sentence(text))
    
    #removing some special charachter  
#    text= re.sub("[\"\+\|\*\?\(\)\/\\\^\[\]``<>\.{}`′’‘';•«»,@:~!\=%&]+", ' ',text)  #except _ -
#    text= re.sub("[\"\“\”\+\-\|\*\?\(\)\/\\\^\[\]\.{}_`′’‘';•«,@:~!\=%&]+", ' ',text) 
    
    #removing hashtag
#    text=re.sub('#', ' ', text) 
    
    #removing numbers not attached to alphabets
    '''
    text=re.sub("(^)(\d+)?(\.)?(\d+)? ",' ',text)   #removing numer at the beginning
    text=re.sub("(\s)[0-9]?(\.)?(\d+) ",' ',text) #py6 and py9
    text= re.sub(" (\.)(\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) (\d+) (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) ", ' ',text)
    text= re.sub(" (\d+)$", ' ',text)
    '''
    #text=re.sub("\S+(\d+) ",' ',text) # alphabet+digit (attached)
    #text=re.sub(" (\d+)\S+",' ',text) # digit+alphabet (attached)
    #text=re.sub(" \S+(\d+)\S+ ",' ',text) # alphabet+digit+alphabet (attached)
    #text=re.sub("(\d+)",' ',text)  #removing any number anywhere but keeps \. for decimal numbers

    #removing space
    text=re.sub('\s+',' ',text)     
    
    text=re.sub('(^)rt ','',text)    # if we do not want to remove stopwords

    text= nltk.word_tokenize(text)
    #text= text.split() #sometimes

    #removing_stopwords 
    #text_without_sw = [word.lower() for word in text if word.lower() not in stopwords.words()] #very slow
    text = [word for word in text if word not in cachedStopWords]

    #lemmatization
    text= [ lemm(word, pos="v") for word in text]
    text= [ lemm(word, pos="n") for word in text]
    text= [ lemm(word, pos="a") for word in text]
    
    #stemming 
    #text = [Stem(word) for word in text]
    
    text=' '.join(text)
    text=re.sub("''",'''"''', text)    #since nltk.tokenize converts second " to ''"
    text=re.sub("``",'''"''', text)   # since nltk.tokenize converts first " to " ``
    
    return text

In [9]:
def cleaningA (text):
    
    #order of lines is important
    
    text=strip_tags(text)
    #text=html.unescape(text)   # stripping or converting html entities 
    #text=saxutils.unescape(text)
    
    #convertings words that their lower and uper cases are different
    text=re.sub(" US | U\.S\. ", ' USA ', text) # before lower 
    
    #converting
    text = re.sub("“|”", '', text)  #before next lines
    text = re.sub("’|′|‘|`", "'", text)  #before next lines
    
    #removing tabs and lines
    text=re.sub('\t|\n', ' ', text)
    
    #converting lower_case
    #text = text.lower() 
    
    #converting
    #text=re.sub('\$|£|€|¥|dollar|dollars|yen|yens|euros', ' money ', text)   # not euro 
    #text=re.sub('Dollar|Dollars|Yen|Yens|Euros', ' money ', text)   # not euro 

    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    
    #removing emoji
    text = emoji_pattern.sub(r' ', text) 

    #removing emojis and non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+|,Ä¶',' ', text)  
    
    #removeing http and https (URL)
    text = re.sub(r'(http://|https://)\S+', '', text)
    text = re.sub(r'(HTTP://|HTTPS://)\S+', '', text)

    #removing www (URL)
    text=re.sub(r'www\.\S+', '', text)
    text=re.sub(r'WWW\.\S+', '', text)

    #removing targets
    text=re.sub('( |^)@\S+', '', text) 

    #removing common expressions
    '''
    text=re.sub("looking forward to|look forward to|make sure|kidding me|\
                |in my opinion|by the way,|as soon as possible|shaking my head|i don't know|I do not know|\
                |in real life|quote of the day|as far as i know|shake my head|\
                |to be honest|in other words|let me know|just kidding|hope that helps|hat tip|\
                |just like that|happy birthday|never mind|well-done|\
                |in my humble opinion|happy new year|you're welcome|you are welcome| \
                |it doesn't matter|it does not matter|i think|i wonder|do you think", ' ', text)  
    '''
    
    #convertings
    text=re.sub("can't", 'cannot', text) # before other n't 
    text=re.sub("can not ", 'cannot ', text)  
    text=re.sub("'ve",' have', text)
    text=re.sub("n't",' not', text)
    text=re.sub("'ll",' will', text)
    #text=re.sub("'d",' would', text)
    text=re.sub("'re",' are', text)
    text=re.sub("i'm",'i am', text)
    #text=re.sub("&",' and ', text)
    text=re.sub(" w/ ",' with ', text)
    text=re.sub(" w/i | w/in ",' within ', text)
    text=re.sub(" w/o ",' without ', text)
    text=re.sub(" c/o ",' care of ', text)
    text=re.sub(" h/t ",' hat tip ', text)
    text=re.sub(" b/c ",' because ', text)
    #text=re.sub("=",' equals to ', text)
    text=re.sub("=",' = ', text)
    #text=re.sub("\+",' + ', text)
    #text=re.sub("\+",' plus ', text)
    text=re.sub("United States",'USA', text)
    text=re.sub("United Kingdom",'UK', text)
    text=re.sub(" the US ",' USA ', text)
    text=re.sub("start-up|start_up",'startup', text)
    text=re.sub("U\.S\.A", 'USA', text)  #try text=re.sub("u.s.a", 'usa', text) with text=substantially 
    text=re.sub("aka", 'also known as', text)     
    text=re.sub("'"," ' ", text)     
    
    text= re.sub("(\?)+", '? ',text)     
    text= re.sub("(!)+", '! ',text)     
    text= re.sub("(\.\.)+", ' ',text)   

#    text = "".join(lemmatize_sentence(text))
    
    #removing some special charachter  
#    text= re.sub("[\"\+\|\*\?\(\)\/\\\^\[\]``<>\.{}`′’‘';•«»,@:~!\=%&]+", ' ',text)  #except _ -
#    text= re.sub("[\"\“\”\+\-\|\*\?\(\)\/\\\^\[\]\.{}_`′’‘';•«,@:~!\=%&]+", ' ',text) 
    
    #removing hashtag
#    text=re.sub('#', ' ', text)  
    
    #removing numbers not attached to alphabets
    '''
    text=re.sub("(^)(\d+)?(\.)?(\d+)? ",' ',text)   #removing numer at the beginning
    text=re.sub("(\s)[0-9]?(\.)?(\d+) ",' ',text) #py6 and py9
    text= re.sub(" (\.)(\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) (\d+) (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) ", ' ',text)
    text= re.sub(" (\d+)$", ' ',text)
    '''
    #text=re.sub("\S+(\d+) ",' ',text) # alphabet+digit (attached)
    #text=re.sub(" (\d+)\S+",' ',text) # digit+alphabet (attached)
    #text=re.sub(" \S+(\d+)\S+ ",' ',text) # alphabet+digit+alphabet (attached)
    #text=re.sub("(\d+)",' ',text)  #removing any number anywhere but keeps \. for decimal numbers

    #removing space
    text=re.sub('\s+',' ',text)     
    
    text=re.sub('(^)RT | RT ','',text)    # if we do not want to remove stopwords

#    text= nltk.word_tokenize(text)
    #text= text.split() #sometimes

    #removing_stopwords 
    #text_without_sw = [word.lower() for word in text if word.lower() not in stopwords.words()] #very slow
#    text = [word for word in text if word not in cachedStopWords]

    #lemmatization
    #text= [ lemm(word, pos="v") for word in text]
    #text= [ lemm(word, pos="n") for word in text]
    #text= [ lemm(word, pos="a") for word in text]
    
    #stemming 
    #text = [Stem(word) for word in text]
    
#    text=' '.join(text)
#    text=re.sub("''",'''"''', text)    #since nltk.tokenize converts second " to ''"
#    text=re.sub("``",'''"''', text)   # since nltk.tokenize converts first " to " ``
    
    return text

In [10]:
def cleaning_punc(text):
    '''
    text=re.sub("looking forward|look forward to|make sure|kidding me|\
                |in my opinion|by the way,|as soon as possible|shaking my head|i don't know|I do not know|\
                |in real life|quote of the day|as far as i know|shake my head|\
                |to be honest|in other words|let me know|just kidding|hope that helps|hat tip|\
                |just like that|happy birthday|never mind|well-done|\
                |in my humble opinion|happy new year|you're welcome|you are welcome| \
                |it doesn't matter|it does not matter|i think|i wonder|do you think", ' ', text) 
    '''
#    text= re.sub("[\"\+\|\*\?\(\)\/\\\^\[\]``<>\.{}`′’‘';•«»,@:~!\=%&]+", ' ',text)  #except _ -
    text= re.sub("[\"\“\”\+\|\*\?\(\)\/\\\^\[\]\.{}`′’‘';•«,@:~!\=%&]+", ' ',text) #except  \- _
    
    #removing hashtag
    text=re.sub('#', ' ', text) 
    
    '''
    #removing numbers not attached to alphabets
    text=re.sub("(^)(\d+)?(\.)?(\d+)? ",' ',text)   #removing numer at the beginning
    text=re.sub("(\s)[0-9]?(\.)?(\d+) ",' ',text) #py6 and py9
    text= re.sub(" (\.)(\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) (\d+) (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) ", ' ',text)
    text= re.sub(" (\d+)$", ' ',text)
    text= re.sub(">", ' ',text)
    text= re.sub("<", ' ',text)
    text= re.sub(" - ", ' ',text)
    text= re.sub(" --", ' ',text)
    '''
    
#    text = "".join(lemmatize_sentence(text))
    
    return text

In [11]:
def cleaning_numbers(text):

    #removing numbers not attached to alphabets
    text=re.sub("(^)(\d+)?(\.)?(\d+)? ",' ',text)   #removing numer at the beginning
    text=re.sub("(\s)[0-9]?(\.)?(\d+) ",' ',text) #py6 and py9
    text= re.sub(" (\.)(\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) (\d+) (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) ", ' ',text)
    text= re.sub(" (\d+)$", ' ',text)
    
    return text

In [12]:
#df_mng = pd.read_csv('/archives1/Datasets/TweetsWorld/mng_tweets_world.csv', delimiter='\t', na_values=".",error_bad_lines=False)#,warn_bad_lines=False)
df_mng0 = pd.read_csv('mng_tweets_world.csv', delimiter='\t', na_values=".",error_bad_lines=False)#,warn_bad_lines=False)
print(df_mng0.shape)
print(df_mng0.columns)
#print(df_mng0.head()) # Preview the first 5 lines of the loaded data 

#rows_mng=list(df_mng0[['user_id', 'tweet','tweet_created_at','location_profile']].itertuples(index=False, name=None)) #rows_ent0
#rows_mng= list(zip(df_mng0.user_id, df_mng.tweet))
#rows_mng=df_mng0[['user_id','tweet']].apply(tuple, axis=1) 
df_mng = df_mng0[['tweet','tweet_created_at']]   


del df_mng0

print("Number of tweets in mng:",len(df_mng))  #rows_mng0

(46502302, 4)
Index(['user_id', 'tweet', 'tweet_created_at', 'location_profile'], dtype='object')
Number of tweets in mng: 46502302


In [13]:
#print('Memory size of mng:',sys.getsizeof(df_mng)) #rows_mng0

In [14]:
'''
df_mng0 = pd.read_csv('mng_tweets_world.csv', delimiter='\t', na_values=".",error_bad_lines=False,warn_bad_lines=False)
#df_mng0 = pd.read_csv('/archives1/Datasets/TweetsWorld/mng_tweets_world.csv', delimiter='\t', na_values=".",error_bad_lines=False,warn_bad_lines=False)
print(df_mng0.shape)
print(df_mng0.columns)
#print(df_mng0.head()) # Preview the first 5 lines of the loaded data 

#rows_mng=list(df_mng0[['user_id', 'tweet','tweet_created_at','location_profile']].itertuples(index=False, name=None)) #rows_mng0
#rows_mng= list(zip(df_mng0.user_id, df_mng.tweet))
#rows_mng=df_mng0[['user_id','tweet']].apply(tuple, axis=1) 
df_mng = df_mng0[['user_id', 'tweet','tweet_created_at','location_profile']]

del df_mng0

print(len(df_mng)) #rows_mng0
print('memry size of mng:', sys.getsizeof(df_mng)) #rows_mng0
'''
pass

In [15]:
#with open("./years/mng_cleaning_classification.txt", "rb") as fp:  
#    mng_tweets_rows=pickle.load(fp)
#len(mng_tweets_rows)

In [16]:
year='2019'
with open("./years/"+year+"_mng_year_ind.txt", "rb") as fp:   
    mng_year_ind_2019=pickle.load(fp)

len(mng_year_ind_2019)

2000000

In [17]:
year='2017'
with open("./years/"+year+"_mng_year_ind.txt", "rb") as fp:   
    mng_year_ind_2017=pickle.load(fp)

len(mng_year_ind_2017)

2000000

In [18]:
year='2019'
with open("./years/"+year+"result_mng_vote.txt", "rb") as fp:  
    vote_19=pickle.load(fp)

with open("./years/"+year+"result_mng_label.txt", "rb") as fp:  
    label_19=pickle.load(fp)

In [19]:
mng_2019_rel=[]
mng_2019_irrel=[]
for i , k in enumerate(mng_year_ind_2019):

    if '2019'  not in df_mng['tweet_created_at'][k]:
        print(k)
    if label_19[i]==1:
        mng_2019_rel.append(k)
    else:
        mng_2019_irrel.append(k)  

In [20]:
len(mng_2019_rel), len(mng_2019_irrel)

(600143, 1399857)

In [21]:
year='2017'
with open("./years/"+year+"result_mng_vote.txt", "rb") as fp:  
    vote_17=pickle.load(fp)

with open("./years/"+year+"result_mng_label.txt", "rb") as fp:  
    label_17=pickle.load(fp)

In [22]:
mng_2017_rel=[]
mng_2017_irrel=[]
for i , k in enumerate(mng_year_ind_2017):
    if '2017' not in df_mng['tweet_created_at'][k]:
        print(k)
    if label_17[i]==1:
        mng_2017_rel.append(k)
    else:
        mng_2017_irrel.append(k)  

In [23]:
len(mng_2017_rel), len(mng_2017_irrel)

(756871, 1243129)

In [24]:
#df_mng['tweet_created_at'][mng_2017_rel[0]], df_mng['tweet'][mng_2017_rel[0]]

In [25]:
ft = fasttext.load_model('cc.en.300.bin')
print(ft.get_dimension())

300


In [26]:
t0=time.time()
embedding_dim=300
max_tokens=50
#X_train_test=[]

tweet_vectors_19=np.zeros([len(mng_2019_rel),embedding_dim],dtype='float32')

for i,index in enumerate(mng_2019_rel):
    tweet=df_mng['tweet'][index]
    
    #tweet='I like it.'
    text=cleaning(tweet) # the result for cleaning and cleaningA is different 
    #text=cleaning_punc(text)
    #text=cleaning_numbers(text)
#train_texts_tokens=[]
#for text in train_texts:
#    tokens=text.split()
    #tokens=nltk.word_tokenize(text)
    tokens=tknzr.tokenize(text)
    #train_texts_tokens.append(tokens)
    
#print(len(train_texts_tokens))

#for kk,tokens in enumerate(train_texts_tokens):
    if tokens !="" and tokens!=' ' and  tokens!=[]:   
        a1=[]
        num_tok=0
        for token in tokens:
            num_tok +=1
            a1.append( ft.get_word_vector(token) )
                    
        #print(a1)
        #print()
        #temp1=np.sum(a1,axis=0)/num_tok
        #print(temp1)
        #print()
        temp=np.mean(a1,axis=0)
        #print(abc-temp1)
        
        
        #if len(tokens)>max_tokens:
        #    temp=a1[0:max_tokens]
        #elif len(tokens)==max_tokens:
        #    temp=a1
        #else: # if len(tokens)<max_tokens:
        #    temp[0:len(tokens)]=a1
        #X_train_test.append(temp) 
        tweet_vectors_19[i]=temp

    else:
        #print(index)
        #print(tweet)
        #print('strange')
        temp=np.zeros([1,embedding_dim])
#        X_train_test.append(temp)
        tweet_vectors_19[i]=temp
        
time.time()-t0


373.6564724445343

In [27]:
np.shape(tweet_vectors_19)

(600143, 300)

In [28]:
type(tweet_vectors_19)

numpy.ndarray

In [29]:
import umap

In [30]:
#X_umap = umap.UMAP().fit_transform(tweet_vectors_19)

In [31]:
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import SparsePCA

from sklearn.manifold import TSNE
t0=time.time()

#kpca = KernelPCA(n_components=2, kernel='linear')
#X_kpca = kpca.fit_transform(tweet_vectors_19)


pca = PCA(n_components=2)
X_pca2019 = pca.fit_transform(tweet_vectors_19)

print(time.time()-t0)

#tsne = TSNE()
#X_tsne = tsne.fit_transform(X_pca)

print(time.time()-t0)

5.077962636947632
5.077962636947632


In [32]:
X_pca2019.shape

(600143, 2)

In [33]:
from sklearn.manifold import Isomap
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.decomposition import TruncatedSVD

In [34]:
#embedding = LocallyLinearEmbedding(n_components=2)
#X_transformed = embedding.fit_transform(tweet_vectors_19)
#X_transformed.shape

In [35]:
'''
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
x_svd=svd.fit_transform(tweet_vectors_19)
print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())
print(svd.explained_variance_ratio_.sum())
x_svd.shape
'''
pass

In [36]:
#embedding = Isomap(n_components=2)
#X_transformed_iso = embedding.fit_transform(tweet_vectors_19)
#X_transformed_iso.shape

In [37]:
'''
from sklearn.manifold import MDS
embedding = MDS(n_components=2)
X_transformed = embedding.fit_transform(tweet_vectors_19)
X_transformed.shape
'''
pass

In [38]:
import hdbscan
clusterer_19 = hdbscan.HDBSCAN(min_cluster_size=500)

In [39]:
t0=time.time()
clusterer_19.fit(X_pca2019)
#clusterer_19.fit(x_svd)
print(time.time()-t0)

63.52549123764038


In [40]:
clusterer_19.labels_
clusterer_19.labels_[0:10]
clusterer_19.labels_.max()
np.unique(clusterer_19.labels_)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
      dtype=int64)

# Number of tweets in each cluster for 2019

In [41]:
cluster_index=[None]*(clusterer_19.labels_.max()+1)
#display(cluster_index)

for i in range(-1,clusterer_19.labels_.max()+1):
    aa1=np.where(clusterer_19.labels_ == i)
    if i!=-1:
        cluster_index[i]=aa1[0]
    print(len(aa1[0]))

print('----')
#len(cluster_index[4]) 

412486
1796
949
1517
1908
572
9295
2110
674
1930
712
1410
1743
1447
1220
1131
3447
516
1195
1537
524
1428
729
855
587
134023
1143
949
975
3223
8112
----


In [42]:
#cluster_index[0][0:10]

In [43]:
stop=cachedStopWords
stop.update(set(english_alghabet))
stop.update(set(numbers_remove))
stop.update(set(['rt','u']))

exclude_punc = set(string.punctuation) 
lemma = WordNetLemmatizer() 

def clean(doc):
    
    doc=cleaning(doc)
    doc=cleaning_punc(doc)
    doc=cleaning_numbers(doc)
    doc= re.sub(" 00am | 00pm | pm | am ", '',doc)
    # convert text into lower case + split into words
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    #stop_free = " ".join([i for i in doc.lower().split() re.sub("(\d+)", '',i))

    stop_free_=[]                      
    for i in stop_free.lower().split():
                          
        if not i.isdigit():
            stop_free_.append(i)
                          
    stop_free = " ".join(stop_free_)

    #text= re.sub(" (\d+) ", ' ',text)
    
    # remove any stop words present
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude_punc)  
    
    # remove punctuations + normalize the text
    cleaned = " ".join(lemma.lemmatize(word) for word in punc_free.split())  
    return cleaned

#  Topics and thier words for 2019

In [44]:
'''
for ii in range(0,clusterer_19.labels_.max()+1):
    print("cluster index : ",ii)

    set_x=[]
    for i in cluster_index[ii]:
        text=df_mng.tweet[mng_2019_rel[i]]
        #print(text)
        #print(df_mng['tweet'][mng_2019_rel[i]])
        #print('')
        set_x.append(text)

    #len(set_x),type(set_x)

    # clean data stored in a new list
    cleaned_set = [clean(doc).split() for doc in set_x] 

    #len(cleaned_set)
    

    # Converting text into numerical representation
    tf_idf_vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False)
    # Converting text into numerical representation
    tf_idf_arr = tf_idf_vectorizer.fit_transform(cleaned_set)
    # Creating vocabulary array which will represent all the corpus 
    vocab_tf_idf = tf_idf_vectorizer.get_feature_names()
    display(len(vocab_tf_idf))


    cv_vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)
    cv_arr = cv_vectorizer.fit_transform(cleaned_set)
    vocab_cv = cv_vectorizer.get_feature_names()
    display(len(vocab_cv))
    
    

    lda_model = LatentDirichletAllocation(n_components = 1, max_iter = 20, random_state = 20)
    X_topics = lda_model.fit_transform(tf_idf_arr)
    topic_words = lda_model.components_
    
    

    #  Define the number of Words that we want to print in every topic : n_top_words
    n_top_words = 21

    for i, topic_dist in enumerate(topic_words):
    
        # np.argsort to sorting an array or a list or the matrix acc to their values
        sorted_topic_dist = np.argsort(topic_dist)
    
        # Next, to view the actual words present in those indexes we can make the use of the vocab created earlier
        topic_words = np.array(vocab_tf_idf)[sorted_topic_dist]
    
        # so using the sorted_topic_indexes we ar extracting the words from the vocabulary
        # obtaining topics + words
        # this topic_words variable contains the Topics  as well as the respective words present in those Topics
        topic_words = topic_words[:-n_top_words:-1]
        print ("Topic", str(i+1), topic_words)
        
        print('----------')
'''
pass

In [45]:
df_mng.tweet[mng_2019_rel[0]]

"Don't miss the new Quotable articles! Subscribe to our bi-weekly newsletter now! https://t.co/53zxj4NU7F #sales https://t.co/4BDfV2cmYg"

In [46]:
set_x=[]
for i in range(0,len(mng_2019_rel)):
    #print(mng_2019_rel[i])
    text=df_mng.tweet[mng_2019_rel[i]]
    #print(text)
    #print(df_mng['tweet'][mng_2019_rel[i]])
    #print('')
    set_x.append(text)

#len(set_x),type(set_x)

# clean data stored in a new list
cleaned_set = [clean(doc).split() for doc in set_x] 

In [47]:
len(cleaned_set)

600143

In [49]:
#len(cleaned_set)


# Converting text into numerical representation
tf_idf_vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False)
# Converting text into numerical representation
tf_idf_arr = tf_idf_vectorizer.fit_transform(cleaned_set)
# Creating vocabulary array which will represent all the corpus 
vocab_tf_idf = tf_idf_vectorizer.get_feature_names()
display(len(vocab_tf_idf))


# Converting text into numerical representation
cv_vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)
# Array from Count Vectorizer 
cv_arr = cv_vectorizer.fit_transform(cleaned_set)
vocab_cv = cv_vectorizer.get_feature_names()
display(len(vocab_cv))

241690

241690

In [60]:
lda_model = LatentDirichletAllocation(n_components = 10, max_iter = 20, random_state = 20)
X_topics = lda_model.fit_transform(tf_idf_arr)
topic_words = lda_model.components_



#  Define the number of Words that we want to print in every topic : n_top_words
n_top_words = 21

for i, topic_dist in enumerate(topic_words):

    # np.argsort to sorting an array or a list or the matrix acc to their values
    sorted_topic_dist = np.argsort(topic_dist)

    # Next, to view the actual words present in those indexes we can make the use of the vocab created earlier
    topic_words = np.array(vocab_tf_idf)[sorted_topic_dist]

    # so using the sorted_topic_indexes we ar extracting the words from the vocabulary
    # obtaining topics + words
    # this topic_words variable contains the Topics  as well as the respective words present in those Topics
    topic_words = topic_words[:-n_top_words:-1]
    print ("Topic", str(i+1), topic_words)

    print('----------')
    

Topic 1 ['check' 'new' 'help' 'list' 'update' 'ticket' 'available' 'work' 'event'
 'feature' 'book' 'free' 'release' 'excite' 'need' 'dm' 'site' 'time'
 'announce' 'launch']
----------
Topic 2 ['post' 'photo' 'new' 'market' 'book' 'email' 'work' 'business' 'content'
 'medium' 'start' 'social' 'link' 'write' 'read' 'blog' 'need' 'tip'
 'hope' 'video']
----------
Topic 3 ['follow' 'win' 'join' 'time' 'giveaway' 'kindly' 'chance' 'money' 'enter'
 'prize' 'incredible' 'new' 'follower' 'event' 'note' 'team' 'twitter'
 'view' 'competition' 'winner']
----------
Topic 4 ['look' 'enjoy' 'christmas' 'fun' 'money' 'plan' 'new' 'visit' 'work'
 'earn' 'coffee' 'london' 'bite' 'xx' 'concern' 'invite' 'time' 'badge'
 'disney' 'level']
----------
Topic 5 ['work' 'brexit' 'support' 'vote' 'labour' 'party' 'uk' 'new' 'deal'
 'tory' 'johnson' 'boris' 'mp' 'eu' 'time' 'people' 'corbyn' 'nh' 'leader'
 'need']
----------
Topic 6 ['agree' 'money' 'data' 'new' 'need' 'business' 'work' 'company' 'market'
 'cha

In [61]:
tf_idf_arr.shape[0]

600143

In [62]:
# To view what topics are assigned to the douments:

doc_topic = lda_model.transform(tf_idf_arr)  
topic_num=dict()

# iterating over ever value till the end value
for n in range(doc_topic.shape[0]):
    
    # argmax() gives maximum index value
    topic_doc = doc_topic[n].argmax()
    
    if topic_doc+1 in topic_num:
        topic_num[topic_doc+1] +=1
    else:
        topic_num[topic_doc+1]=1
        
        
    
    # document is n+1  
    #print ("Document", n+1, " -- Topic:" ,topic_doc)

topic_num

{2: 81108,
 6: 125915,
 7: 62446,
 5: 39735,
 10: 96740,
 1: 45934,
 4: 29836,
 3: 29933,
 8: 42318,
 9: 46178}

In [63]:
from sklearn.cluster import OPTICS
from sklearn.cluster import DBSCAN

In [64]:
#clusterer_optics = OPTICS(min_samples=5).fit(X_pca)
#clusterer_dbscan = DBSCAN(eps=5, min_samples=2).fit(X_pca)

In [65]:
from sklearn.cluster import KMeans
#kmeans = KMeans(n_clusters=3) 
#mng_kmeans_clustering = kmeans.fit(A)

In [67]:
t0=time.time()
embedding_dim=300
max_tokens=50
#X_train_test=[]

tweet_vectors_17=np.zeros([len(mng_2017_rel),embedding_dim],dtype='float32')

for i,index in enumerate(mng_2017_rel):
    tweet=df_mng['tweet'][index]
    
    #tweet='I like it.'
    text=cleaning(tweet) # the result for cleaning and cleaningA is different 
    #text=cleaning_punc(text)
    #text=cleaning_numbers(text)
#train_texts_tokens=[]
#for text in train_texts:
#    tokens=text.split()
    #tokens=nltk.word_tokenize(text)
    tokens=tknzr.tokenize(text)
    #train_texts_tokens.append(tokens)
    
#print(len(train_texts_tokens))

#for kk,tokens in enumerate(train_texts_tokens):
    if tokens !="" and tokens!=' ' and  tokens!=[]:   
        a1=[]
        num_tok=0
        for token in tokens:
            num_tok +=1
            a1.append( ft.get_word_vector(token) )
                    
        #print(a1)
        #print()
        #temp1=np.sum(a1,axis=0)/num_tok
        #print(temp1)
        #print()
        temp=np.mean(a1,axis=0)
        #print(abc-temp1)
        
        
        #if len(tokens)>max_tokens:
        #    temp=a1[0:max_tokens]
        #elif len(tokens)==max_tokens:
        #    temp=a1
        #else: # if len(tokens)<max_tokens:
        #    temp[0:len(tokens)]=a1
        #X_train_test.append(temp) 
        tweet_vectors_17[i]=temp

    else:
        #print(index)
        #print(tweet)
        #print('strange')
        temp=np.zeros([1,embedding_dim])
#        X_train_test.append(temp)
        tweet_vectors_17[i]=temp
        
time.time()-t0


483.87185645103455

In [68]:
np.shape(tweet_vectors_17)

(756871, 300)

In [69]:
t0=time.time()

pca = PCA(n_components=2)
X_pca2017 = pca.fit_transform(tweet_vectors_17)

print(time.time()-t0)

5.45512056350708


In [70]:
import hdbscan

clusterer_17 = hdbscan.HDBSCAN(min_cluster_size=500)

In [71]:
t0=time.time()
clusterer_17.fit(X_pca2017)
#clusterer.fit(x_svd)
print(time.time()-t0)

87.95303344726562


In [72]:
clusterer_17.labels_

clusterer_17.labels_[0:10]

clusterer_17.labels_.max()

np.unique(clusterer_17.labels_)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,
       33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
       50], dtype=int64)

# Number of tweets in each cluster for 2017

In [73]:
cluster_index=[None]*(clusterer_17.labels_.max()+1)
#display(cluster_index)

for i in range(-1,clusterer_17.labels_.max()+1):
    aa1=np.where(clusterer_17.labels_ == i)
    if i!=-1:
        cluster_index[i]=aa1[0]
    print(len(aa1[0]))

print('----')
#len(cluster_index[4]) 

458031
910
585
830
990
816
855
718
504
3852
1382
1200
2221
906
3532
554
3100
7508
12210
799
1663
736
1095
691
568
1382
754
1150
517
1541
767
569
1527
2616
618
655
563
1334
994
509
1130
515
1311
6985
2295
208232
2003
1272
1041
658
8337
1340
----


# Topics and thier words for 2017

In [74]:
'''
for ii in range(0,clusterer_17.labels_.max()+1):
    print("cluster index : ",ii)

    set_x=[]
    for i in cluster_index[ii]:
        text=df_mng.tweet[mng_2017_rel[i]]
        #print(text)
        #print(df_mng['tweet'][mng_2017_rel[i]])
        #print('')
        set_x.append(text)

    #len(set_x),type(set_x)

    # clean data stored in a new list
    cleaned_set = [clean(doc).split() for doc in set_x] 

    #len(cleaned_set)
    

    # Converting text into numerical representation
    tf_idf_vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False)
    # Converting text into numerical representation
    tf_idf_arr = tf_idf_vectorizer.fit_transform(cleaned_set)
    # Creating vocabulary array which will represent all the corpus 
    vocab_tf_idf = tf_idf_vectorizer.get_feature_names()
    display(len(vocab_tf_idf))


    cv_vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)
    cv_arr = cv_vectorizer.fit_transform(cleaned_set)
    vocab_cv = cv_vectorizer.get_feature_names()
    display(len(vocab_cv))
    
    

    lda_model = LatentDirichletAllocation(n_components = 1, max_iter = 20, random_state = 20)
    X_topics = lda_model.fit_transform(tf_idf_arr)
    topic_words = lda_model.components_
    
    

    #  Define the number of Words that we want to print in every topic : n_top_words
    n_top_words = 21

    for i, topic_dist in enumerate(topic_words):
    
        # np.argsort to sorting an array or a list or the matrix acc to their values
        sorted_topic_dist = np.argsort(topic_dist)
    
        # Next, to view the actual words present in those indexes we can make the use of the vocab created earlier
        topic_words = np.array(vocab_tf_idf)[sorted_topic_dist]
    
        # so using the sorted_topic_indexes we ar extracting the words from the vocabulary
        # obtaining topics + words
        # this topic_words variable contains the Topics  as well as the respective words present in those Topics
        topic_words = topic_words[:-n_top_words:-1]
        print ("Topic", str(i+1), topic_words)
        
        print('----------')
'''
pass

In [75]:


    set_x=[]
    for i in range(0,len(mng_2017_rel)):
        #print(mng_2017_rel[i])
        text=df_mng.tweet[mng_2017_rel[i]]
        #print(text)
        #print(df_mng['tweet'][mng_2017_rel[i]])
        #print('')
        set_x.append(text)

    #len(set_x),type(set_x)

    # clean data stored in a new list
    cleaned_set = [clean(doc).split() for doc in set_x] 

    #len(cleaned_set)
    

    # Converting text into numerical representation
    tf_idf_vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False)
    # Converting text into numerical representation
    tf_idf_arr = tf_idf_vectorizer.fit_transform(cleaned_set)
    # Creating vocabulary array which will represent all the corpus 
    vocab_tf_idf = tf_idf_vectorizer.get_feature_names()
    display(len(vocab_tf_idf))


    cv_vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)
    cv_arr = cv_vectorizer.fit_transform(cleaned_set)
    vocab_cv = cv_vectorizer.get_feature_names()
    display(len(vocab_cv))
    
    

    lda_model = LatentDirichletAllocation(n_components = 10, max_iter = 20, random_state = 20)
    X_topics = lda_model.fit_transform(tf_idf_arr)
    topic_words = lda_model.components_
    
    

    #  Define the number of Words that we want to print in every topic : n_top_words
    n_top_words = 21

    for i, topic_dist in enumerate(topic_words):
    
        # np.argsort to sorting an array or a list or the matrix acc to their values
        sorted_topic_dist = np.argsort(topic_dist)
    
        # Next, to view the actual words present in those indexes we can make the use of the vocab created earlier
        topic_words = np.array(vocab_tf_idf)[sorted_topic_dist]
    
        # so using the sorted_topic_indexes we ar extracting the words from the vocabulary
        # obtaining topics + words
        # this topic_words variable contains the Topics  as well as the respective words present in those Topics
        topic_words = topic_words[:-n_top_words:-1]
        print ("Topic", str(i+1), topic_words)
        
        print('----------')
    

255756

255756

Topic 1 ['learn' 'business' 'data' 'new' 'cloud' 'work' 'need' 'customer' 'way'
 'service' 'help' 'build' 'digital' 'technology' 'management' 'change'
 'company' 'look' 'experience' 'http']
----------
Topic 2 ['trump' 'usa' 'uk' 'new' 'work' 'president' 'sign' 'http' 'london'
 'money' 'report' 'world' 'deal' 'country' 'brexit' 'news' 'time'
 'problem' 'car' 'need']
----------
Topic 3 ['work' 'job' 'enjoy' 'check' 'team' 'nice' 'hope' 'new' 'start' 'time'
 'plan' 'glad' 'hard' 'join' 'meet' 'love' 'interview' 'club' 'help' 'set']
----------
Topic 4 ['money' 'market' 'price' 'new' 'growth' 'rat' 'invest' 'tax' 'investment'
 'home' 'course' 'work' 'rise' 'time' 'investor' 'increase' 'high' 'stock'
 'help' 'mortgage']
----------
Topic 5 ['luck' 'new' 'meet' 'work' 'view' 'team' 'travel' 'celebrate' 'step' 'xx'
 'mile' 'support' 'event' 'award' 'join' 'weather' 'time' 'fitbit' 'area'
 'start']
----------
Topic 6 ['agree' 'work' 'share' 'people' 'vote' 'support' 'need' 'new' 'health'
 'help'

In [76]:
# To view what topics are assigned to the douments:

doc_topic = lda_model.transform(tf_idf_arr)  
topic_num=dict()

# iterating over ever value till the end value
for n in range(doc_topic.shape[0]):
    
    # argmax() gives maximum index value
    topic_doc = doc_topic[n].argmax()
    
    if topic_doc+1 in topic_num:
        topic_num[topic_doc+1] +=1
    else:
        topic_num[topic_doc+1]=1
        
        
    
    # document is n+1  
    #print ("Document", n+1, " -- Topic:" ,topic_doc)

topic_num

{1: 142802,
 9: 57913,
 3: 61032,
 10: 49447,
 7: 83034,
 8: 84770,
 2: 72137,
 6: 98923,
 4: 61210,
 5: 45603}

# Comment codes (below)

In [58]:
'''
def noun_phrases(text):
    # noun phrase by nltk
    sentences = nltk.sent_tokenize(text)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]

    grammar = """NP: {<DT>?<JJ>*<NN.*>+}"""

    cp = nltk.RegexpParser(grammar)
    #for i in sentences:
    #    result = cp.parse(i)
    #    print(result)
    #    print(type(result))
        #result.draw() 
   # noun_phrases_list = [[' '.join(leaf[0].lower() for leaf in tree.leaves()) 

    noun_phrases_list = [[' '.join(leaf[0] for leaf in tree.leaves()) 
                          for tree in cp.parse(sent).subtrees() 
                          if tree.label()=='NP'] 
                          for sent in sentences]


    phrases_all=[]
    for i in noun_phrases_list:
        phrases_all.extend(i)
    
    return phrases_all
'''
pass

In [59]:
'''
def noun_phrases1(text):
    # noun phrase by Spacy
    doc = nlp(text)
    
    phrases_all=[]
    for phrase in doc.noun_chunks:
        phrases_all.append(phrase.text)
        #phrases_all.append(phrase.text.lower())

    return phrases_all
'''
pass

In [60]:
'''
with open("./mng_results300_np.txt", "rb") as fp:   
    mng_results = pickle.load(fp)

num_mng_positive=0
for i in mng_results:
    if i>=0.5:
        num_mng_positive +=1

print("Number of detected relevant tweets to mng:",num_mng_positive)
'''
pass

In [61]:
'''
with open("./mng_results300.txt", "rb") as fp:   
    mng_results = pickle.load(fp)

num_mng_positive=0
for i in mng_results:
    if i>=0.5:
        num_mng_positive +=1

print("Number of detected relevant tweets to mng:", num_mng_positive)
'''
pass

# 100 most frequnent hashtags in MNG (with thier frequencies )

In [62]:
'''
t0 = time.time()
mng_hash_2017=[]
for i,value in enumerate(mng_results):
    if value>=0.5:
        text=cleaningA(df_mng['tweet'][i])
        #text= re.sub("[\"\“\”\+\|\*\?\(\)\/\\\^\[\]\.{}`′’‘';•«,@:~!\=%&]+", '',text) #except  \- _

        hashtags = [i[1:].lower()  for i in text.split() if i.startswith("#") ]
        mng_hash_2017.extend(hashtags)
        
mng_Hashtags=dict()

for ph in mng_hash:
    if ph !='':
        if ph not in mng_Hashtags:
            mng_Hashtags[ph]=1
        else:
            mng_Hashtags[ph]+=1  

        
mng_Hashtags = sorted(mng_Hashtags.items(), key=operator.itemgetter(1), reverse=True) 

print( time.time() - t0) 
'''
pass

In [78]:
'''
t0 = time.time()
mng_hash_2017=[]

for i in range(0,len(mng_2017_rel)):
    #print(mng_2017_rel[i])
    text=df_mng.tweet[mng_2017_rel[i]]
    
    text= re.sub("[\"\“\”\+\|\*\?\(\)\/\\\^\[\]\.{}`′’‘';•«,@:~!\=%&]+", '',text) #except  \- _

    text=re.sub("…", '' ,text)
    text= re.sub("(\?)+", '? ',text)     
    text= re.sub("(!)+", '! ',text)     
    text= re.sub("(\.)+", ' ',text)  
    
    hashtags = [i[1:].lower()  for i in text.split() if i.startswith("#") ]
    mng_hash_2017.extend(hashtags)
'''    
'''
for i,value in enumerate(mng_year_ind_2017):
    if label_17[i]==1:
        text=cleaningA(df_mng['tweet'][value])
        text= re.sub("[\"\“\”\+\|\*\?\(\)\/\\\^\[\]\.{}`′’‘';•«,@:~!\=%&]+", '',text) #except  \- _

        hashtags = [i[1:].lower()  for i in text.split() if i.startswith("#") ]
        mng_hash_2017.extend(hashtags)
'''
'''        
        
mng_Hashtags_2017=dict()

for ph in mng_hash_2017:
    if ph !='':
        if ph not in mng_Hashtags_2017:
            mng_Hashtags_2017[ph]=1
        else:
            mng_Hashtags_2017[ph]+=1  

        
mng_Hashtags_2017 = sorted(mng_Hashtags_2017.items(), key=operator.itemgetter(1), reverse=True) 

print( time.time() - t0) 

mng_Hashtags_2017[0:100]
'''
pass

In [64]:
'''
df_hash_mng=pd.DataFrame(mng_Hashtags_2017)
df_hash_mng.columns =['Hashtags in mng','frequency']
df_hash_mng=df_hash_mng.head(100)
df_hash_mng.to_csv('hashtags_mng.csv')
df_hash_mng_styled = df_hash_mng.style 
dfi.export(df_hash_mng_styled,"hashtags_mng.png")
'''
pass

In [79]:
'''
t0 = time.time()
mng_hash_2019=[]

for i in range(0,len(mng_2019_rel)):
    #print(mng_2019_rel[i])
    text=df_mng.tweet[mng_2019_rel[i]]
    
    text= re.sub("[\"\“\”\+\|\*\?\(\)\/\\\^\[\]\.{}`′’‘';•«,@:~!\=%&]+", '',text) #except  \- _

    text=re.sub("…", '' ,text)
    text= re.sub("(\?)+", '? ',text)     
    text= re.sub("(!)+", '! ',text)     
    text= re.sub("(\.)+", ' ',text)  
    
    hashtags = [i[1:].lower()  for i in text.split() if i.startswith("#") ]
    mng_hash_2019.extend(hashtags)

'''
'''
for i,value in enumerate(mng_year_ind_2019):
    if label_19[i]==1:
        text=cleaningA(df_mng['tweet'][value])
        text= re.sub("[\"\“\”\+\|\*\?\(\)\/\\\^\[\]\.{}`′’‘';•«,@:~!\=%&]+", '',text) #except  \- _

        hashtags = [i[1:].lower()  for i in text.split() if i.startswith("#") ]
        mng_hash_2019.extend(hashtags)
'''        
'''        
        
mng_Hashtags_2019=dict()

for ph in mng_hash_2019:
    if ph !='':
        if ph not in mng_Hashtags_2019:
            mng_Hashtags_2019[ph]=1
        else:
            mng_Hashtags_2019[ph]+=1  

        
mng_Hashtags_2019 = sorted(mng_Hashtags_2019.items(), key=operator.itemgetter(1), reverse=True) 

print( time.time() - t0) 

mng_Hashtags_2019[0:100]
'''
pass

# 100 most frequent hashtags in MNG

In [66]:
'''
t0 = time.time()
mng_hash=[]

for i in range(0,len(mng_2017_rel)):
    #print(mng_2017_rel[i])
    text=df_mng.tweet[mng_2017_rel[i]]
    #text= re.sub("[\"\“\”\+\|\*\?\(\)\/\\\^\[\]\.{}`′’‘';•«,@:~!\=%&]+", '',text) #except  \- _
    hashtags = [i[1:].lower()  for i in text.split() if i.startswith("#") ]
    mng_hash.extend(hashtags)
    
    
for i,value in enumerate(mng_results):
    if value>=0.5:
        text=cleaningA(df_mng['tweet'][i])
        #text= re.sub("[\"\“\”\+\|\*\?\(\)\/\\\^\[\]\.{}`′’‘';•«,@:~!\=%&]+", '',text) #except  \- _

        hashtags = [i[1:].lower()  for i in text.split() if i.startswith("#") ]

        mng_hash.extend(hashtags)
    
mng_Hashtags=dict()

for ph in mng_hash:
    if ph != '':
        if ph not in mng_Hashtags:
            mng_Hashtags[ph]=1
        else:
            mng_Hashtags[ph]+=1  

mng_Hashtags = sorted(mng_Hashtags.items(), key=operator.itemgetter(1), reverse=True) 

print( time.time() - t0)
'''
pass

In [67]:
#mng_Hashtags[0:100]

In [68]:
'''
df_hash_mng=pd.DataFrame(mng_Hashtags)
df_hash_mng.columns =['Hashtags in mng','frequency']
df_hash_mng=df_hash_mng.head(100)

df_hash_mng.to_csv('hashtags_mng.csv')

df_hash_mng_styled = df_hash_mng.style 
dfi.export(df_hash_mng_styled,"hashtags_mng.png")
'''
pass

# ignore below lines

In [69]:
'''
t0 = time.time()
# finding concepts by spacy 
mng_spacy=[]
for i,value in enumerate(mng_year_ind_2017):
    if value>=0.5:
        text=cleaningA(df_mng['tweet'][i])
        text=re.sub('#', ' ', text)
#        print(text)
#        print('----')
        concepts_x=noun_phrases1(text)
    
        concepts=[]
        for concept in concepts_x:
            if concept not in cachedStopWords:
                concept1= re.sub('^a |^an |^the ', '', concept)
                concepts.append(concept1)
                
#        print(concepts)
#        print('++++++++++++++++++++++++++++')

        mng_spacy.extend(concepts)
    

mng_noun_phrase_spacy=dict()

for ph in mng_spacy:
    if ph not in mng_noun_phrase_spacy:
        mng_noun_phrase_spacy[ph]=1
    else:
        mng_noun_phrase_spacy[ph]+=1   
            
mng_noun_phrase_spacy = sorted(mng_noun_phrase_spacy.items(), key=operator.itemgetter(1), reverse=True) 
print( time.time() - t0)
'''
pass

In [70]:
'''
t0 = time.time()
# finding concepts by spacy 
mng_spacy=[]
for i,value in enumerate(mng_results):
    if value>=0.5:
        text=cleaningA(df_mng['tweet'][i])
        text=re.sub('#', ' ', text)
#        print(text)
#        print('----')
        concepts_x=noun_phrases1(text)
        
        concepts=[]
        for concept in concepts_x:
            if concept not in cachedStopWords:
                concept1= re.sub('^a |^an |^the ', '', concept)
                concepts.append(concept1)
                
#        print(concepts)
#        print('++++++++++++++++++++++++++++')

        mng_spacy.extend(concepts)
    
    
mng_noun_phrase_spacy=dict()

for ph in mng_spacy:
    if ph not in mng_noun_phrase_spacy:
        mng_noun_phrase_spacy[ph]=1
    else:
        mng_noun_phrase_spacy[ph]+=1   
            
mng_noun_phrase_spacy = sorted(mng_noun_phrase_spacy.items(), key=operator.itemgetter(1), reverse=True) 
print( time.time() - t0)
'''
pass

# First 100 noun phrases in MNG found by NLTK

In [71]:
#t0 = time.time()
## finding concepts by nltk
#mng_2017_nltk=[]

#mng_noun_phrase_2017_nltk=dict()

#for i,value in enumerate(mng_year_ind_2017):
#    if label_17[i]==1:
#        text=cleaningA(df_mng['tweet'][value])
##        print(text)
##        print('----')
#        #text=re.sub('#', ' ', text)
#        concepts_x=noun_phrases(text)
    
#        concepts=[]
#        for concept in concepts_x:
#            if concept not in cachedStopWords:
#                concept= re.sub('^a |^an |^the ', '', concept)
#                concept= re.sub("[\"\“\”\+\|\*\?\(\)\/\\\^\[\]\.{}`′’‘'><;•«,@:~!\=%&]+", '',concept)
#                if concept !='':
#                    if concept not in mng_noun_phrase_2017_nltk:
#                        mng_noun_phrase_2017_nltk[concept]=1
#                    else:
#                        mng_noun_phrase_2017_nltk[concept]+=1
                
#                #concepts.append(concept)
                
##        print(concepts)
##        print('++++++++++++++++++++++++++++')

#        #mng_2017_nltk.extend(concepts)
    

'''
for word in mng_2017_nltk:
    if word !='':
        if word not in mng_noun_phrase_2017_nltk:
            mng_noun_phrase_2017_nltk[word]=1
        else:
            mng_noun_phrase_2017_nltk[word]+=1   
'''

#mng_noun_phrase_2017_nltk = sorted(mng_noun_phrase_2017_nltk.items(), key=operator.itemgetter(1), reverse=True) 
#print( time.time() - t0)
pass

In [72]:
#mng_noun_phrase_2017_nltk[0:300]

In [73]:
'''
for i,value in enumerate(mng_year_ind_2017[0:1000000]):
    if label_17[i]==1:
        text=cleaningA(df_mng['tweet'][value])
        #text= re.sub("#", '',text0)
        if 'Great ' in text:
            if 'Great' in noun_phrases(text):
                
                print(text)
                print(noun_phrases(text))
                print('-----')
                print(noun_phrases1(text))
                print()
'''
pass

In [74]:
#mng_noun_phrase_2017_nltk_non[0:100]

In [75]:
'''
yes_17=[]
for i in mng_noun_phrase_2017_nltk[0:50]:
    yes_17.append(i[0])

non_17=[]
for i in mng_noun_phrase_2017_nltk_non[0:50]:
    non_17.append(i[0])
A
aaa=set(yes_17)-set(non_17)
aaa
'''
pass

In [76]:
'''
ft = fasttext.load_model('cc.en.300.bin')
print(ft.get_dimension())

aaa=list(set(yes)-set(non))
aaa=list(set(yes))
concept_vec=np.zeros((len(aaa),300))

for i,concept in enumerate(aaa):
    concept_vec[i]=ft.get_word_vector(concept)


np.shape(concept_vec)

from sklearn.cluster import OPTICS
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

clustering_dbscan = DBSCAN(eps=5, min_samples=2).fit(concept_vec)
print(clustering_dbscan.labels_)

clustering_optics = OPTICS(min_samples=3).fit(concept_vec)
print(clustering_optics.labels_)

kmeans = KMeans(n_clusters=2) 
mng_kmeans_clustering = kmeans.fit(concept_vec)
print(mng_kmeans_clustering.labels_)
'''
pass

In [77]:
'''
df_ph_mng=pd.DataFrame(mng_noun_phrase_nltk)
df_ph_mng.columns =['noun_phrase in mng by NLTK','frequency']
df_ph_mng=df_ph_mng.head(100)
df_ph_mng.to_csv('noun_phrase_mng.csv')
df_ph_mng_styled = df_ph_mng.style 
dfi.export(df_ph_mng_styled,"noun_phrase_mng_nltk.png")
'''
pass

In [78]:
'''
yes_19=[]
for i in mng_noun_phrase_2019_nltk[0:50]:
    yes_19.append(i[0])

non_19=[]
for i in mng_noun_phrase_2019_nltk_non[0:50]:
    non_19.append(i[0])

bbb=set(yes_19)-set(non_19)
bbb
'''
pass

# First 100 noun phrases in MNG found by NLTK

In [79]:
'''
t0 = time.time()
#finding concepts by nltk 
mng_nltk=[]
for i,value in enumerate(mng_results):
    if value>=0.5:
        text=cleaningA(df_mng['tweet'][i])
        #text=re.sub('#', ' ', text)
#        print(text)
#        print('----')
        concepts_x=noun_phrases(text)
    
        concepts=[]
        for concept in concepts_x:
            if concept not in cachedStopWords:
                #concept= re.sub('^a |^an |^the ', '', concept)
                #concept= re.sub("[\"\“\”\+\|\*\?\(\)\/\\\^\[\]\.{}`′’‘'><;•«,@:~!\=%&]+", '',concept)
                concepts.append(concept)
        
#        print(concepts)       
#        print('++++++++++++++++++++++++++++')
        mng_nltk.extend(concepts)
    

mng_noun_phrase_nltk=dict()

for word in mng_nltk:
    if word !='':
        if word not in mng_noun_phrase_nltk:
            mng_noun_phrase_nltk[word]=1
        else:
            mng_noun_phrase_nltk[word]+=1   
            
mng_noun_phrase_nltk = sorted(mng_noun_phrase_nltk.items(), key=operator.itemgetter(1), reverse=True) 
print( time.time() - t0)
'''
pass

In [80]:
#mng_noun_phrase_nltk[0:100]

In [81]:
'''
df_ph_mng=pd.DataFrame(mng_noun_phrase_nltk)
df_ph_mng.columns =['noun_phrase in mng by NLTK','frequency']
df_ph_mng=df_ph_mng.head(100)

df_ph_mng.to_csv('noun_phrase_mng.csv')

df_ph_mng_styled = df_ph_mng.style 
dfi.export(df_ph_mng_styled,"noun_phrase_mng_nltk.png")
'''
pass

# First 100 (nonstop) words in MNG

In [82]:
'''
t0=time.time()

words_mng=dict()
Sentences=[]
for i, value in enumerate(mng_year_ind_2017):
    if label_17[i]==1:
        text=cleaningA(df_mng['tweet'][value])
        #text=cleaning_punc(text.lower())
        #Sentences.append(text)
        text=text.lower()
        words=text.split()
        
        words_nostop=[]
        for word in words:
            if word not in cachedStopWords:
                word= re.sub("[\"\“\”\+\|\*\?\(\)\/\\\^\[\]\.{}`′’‘'><;•«,@:~!\=%&]+", '',word)
                words_nostop.append(word)
                
    
        for word in words_nostop:
            #word= lemm(wordx)
            if word not in words_mng:
                words_mng[word]=1
            else:
                words_mng[word] +=1
            
words_mng = sorted(words_mng.items(), key=operator.itemgetter(1), reverse=True) 
print( time.time() - t0)
print('number of words:',len(words_mng))

print('First 100 frequent words in mng:')
'''
pass

In [83]:
#words_mng[0:100]

In [84]:
'''
df_word_mng=pd.DataFrame(words_mng)
df_word_mng.columns =['word in mng','frequency']
df_word_mng=df_word_mng.head(100)
df_word_mng.to_csv('word_mng.csv')
df_word_mng_styled = df_word_mng.style 
dfi.export(df_word_mng_styled,"words_mng.png")
'''
pass

# First 100 (nonstop) words in MNG

In [85]:
'''
t0=time.time()

words_mng=dict()
Sentences=[]
for i, value in enumerate(mng_results):
    if value>=0.5:
        text=cleaningA(df_mng['tweet'][i])
        #text=cleaning_punc(text.lower())
        #Sentences.append(text)
        text=text.lower()
        words=text.split()
        
        words_nostop=[]
        for word in words:
            if word not in cachedStopWords:
                words_nostop.append(word)
                    
        for word in words_nostop:
            #word=lemm(wordx)
            if word not in words_mng:
                words_mng[word]=1
            else:
                words_mng[word] +=1
                
words_mng = sorted(words_mng.items(), key=operator.itemgetter(1), reverse=True) 
print( time.time() - t0)
print('number of words:',len(words_mng))
'''
pass

In [86]:
#print('First 100 frequent words in mng:')
#words_mng[0:100]

In [87]:
'''
df_word_mng=pd.DataFrame(words_mng)
df_word_mng.columns =['word in mng','frequency']
df_word_mng=df_word_mng.head(100)

df_word_mng.to_csv('word_mng.csv')

df_word_mng_styled = df_word_mng.style 
dfi.export(df_word_mng_styled,"words_mng.png")
'''
pass

# First 100 bigrams (without stopwords) in MNG

In [88]:
'''
t0=time.time()

bigrams_mng=dict()

for i, value in enumerate(mng_year_ind_2017):
    if label_17[i]==1:
        text=cleaningA(df_mng['tweet'][i])
        #text=cleaning_punc(text.lower())
        text=text.lower()
        words=text.split()
        
        words_nostop=[]
        for word in words:
            if word not in cachedStopWords:
                words_nostop.append(word)
                
        output = list(ngrams(words_nostop, 2))
        
        for combi in output:
            #w0=lemm(combix[0])
            #w1=lemm(combix[1])
            #combi=(w0,w1)
            if combi not in bigrams_mng:
                bigrams_mng[combi]=1
            else:
                bigrams_mng[combi] +=1
                
bigrams_mng = sorted(bigrams_mng.items(), key=operator.itemgetter(1), reverse=True) 
print( time.time() - t0)
print('number of bigrams:',len(bigrams_mng))

print('first 100 frequent bigrams in mng:')

bigrams_mng[0:100]
'''
pass

In [89]:
'''
df_bigram_mng=pd.DataFrame(bigrams_mng)
df_bigram_mng.columns =['bigram in mng','frequency']
df_bigram_mng=df_bigram_mng.head(100)
df_bigram_mng.to_csv('bigram_mng.csv')
df_bigram_mng_styled = df_bigram_mng.style 
dfi.export(df_bigram_mng_styled,"bigrams_mng.png")
'''
pass

# First 100 bigrams (without stopwords) in MNG

In [90]:
'''
t0=time.time()

bigrams_mng=dict()

for i, value in enumerate(mng_results):
    if value>=0.5:
        text=cleaningA(df_mng['tweet'][i])
        #text=cleaning_punc(text.lower())
        words=text.split()
        
        words_nostop=[]
        for word in words:
            if word not in cachedStopWords:
                words_nostop.append(word)
                
        output = list(ngrams(words_nostop, 2))
        
        for combi in output:
            #w0=lemm(combix[0])
            #w1=lemm(combix[1])
            #combi=(w0,w1)
            if combi not in bigrams_mng:
                bigrams_mng[combi]=1
            else:
                bigrams_mng[combi] +=1
                
bigrams_mng = sorted(bigrams_mng.items(), key=operator.itemgetter(1), reverse=True) 
print( time.time() - t0)
print('number of bigrams:',len(bigrams_mng))
'''
pass

In [91]:
#print('first 100 frequent bigrams in mng:')
#bigrams_mng[0:100]

In [92]:
'''
df_bigram_mng=pd.DataFrame(bigrams_mng)
df_bigram_mng.columns =['bigram in mng','frequency']
df_bigram_mng=df_bigram_mng.head(100)

df_bigram_mng.to_csv('bigram_mng.csv')

df_bigram_mng_styled = df_bigram_mng.style 
dfi.export(df_bigram_mng_styled,"bigrams_mng.png")
'''
pass

In [93]:
#df_bigram_mng_styled.to_excel('jjdsfhj.xlsx')

# First 100 trigrams (without stopwords) in MNG

In [94]:
'''
t0=time.time()

trigrams_mng=dict()

for i, value in enumerate(mng_results):
    if value>=0.5:
        text=cleaningA(df_mng['tweet'][i])
        #text=cleaning_punc(text.lower())
        text=text.lower()
        words=text.split()
        
        words_nostop=[]
        for word in words:
            if word not in cachedStopWords:
                words_nostop.append(word)
                
        output = list(ngrams(words_nostop, 3))
        
        for combi in output:
            #w0=lemm(combix[0])
            #w1=lemm(combix[1])
            #w2=lemm(combix[2])
            #combi=(w0,w1,w2)  
            if combi not in trigrams_mng:
                trigrams_mng[combi]=1
            else:
                trigrams_mng[combi] +=1
                
trigrams_mng = sorted(trigrams_mng.items(), key=operator.itemgetter(1), reverse=True) 
print( time.time() - t0)
print('Number of trigrams:',len(trigrams_mng))

print('first 100 frequent trigrams in mng:')
trigrams_mng[0:100]
'''
pass

# First 100 trigrams (without stopwords) in MNG

In [95]:
'''
t0=time.time()

trigrams_mng=dict()

for i, value in enumerate(mng_results):
    if value>=0.5:
        text=cleaningA(df_mng['tweet'][i])
        #text=cleaning_punc(text.lower())
        text=text.lower()
        words=text.split()
        
        words_nostop=[]
        for word in words:
            if word not in cachedStopWords:
                words_nostop.append(word)
                
        output = list(ngrams(words_nostop, 3))
        
        for combi in output:
            #w0=lemm(combix[0])
            #w1=lemm(combix[1])
            #w2=lemm(combix[2])
            #combi=(w0,w1,w2)
            if combi not in trigrams_mng:
                trigrams_mng[combi]=1
            else:
                trigrams_mng[combi] +=1
                
trigrams_mng = sorted(trigrams_mng.items(), key=operator.itemgetter(1), reverse=True) 
print( time.time() - t0)
print('number of trigrams:',len(trigrams_mng))
'''
pass

In [96]:
#print('first 100 frequent trigrams in mng:')
#trigrams_mng[0:100]